In [ ]:
import pandas as pd

# Extracting Transcript

In [ ]:
dataset = pd.read_csv("complaint3.csv")
dataset = dataset.loc[dataset["Video Link"].notnull()]

In [ ]:
from preprocess_data import GenerateTranscript
gtr = GenerateTranscript()
transcripts = gtr.process_videos()

In [ ]:
transcripts.items()

In [ ]:
for idx, transcript in transcripts.items():
    with open(f"transcripts/temp_{idx}.srt", "w") as text_file:
        text_file.write(transcript)

In [ ]:
df = pd.read_csv("./new_data.csv", index_col="video index")

In [ ]:
df_input = df["video link"]
input_dict = df_input.to_dict()

In [ ]:
from preprocess_data import GenerateTranscript

In [ ]:
df_entity = {"video index":[], "product":[], "product name":[], "video title":[], "video link":[], "time stamp":[],	"aspect":[], "complaint label":[], "transcript":[]}

In [ ]:
for index, row in df.iterrows():
    for i in range(1, 7):
        if pd.isna(row[f"aspect {i}"]):
            continue

        for key in df_entity.keys():
            if key in ['time stamp', 'aspect', 'complaint label', 'transcript']:
                df_entity[key].append(row[f"{key} {i}"])
            else:
                df_entity[key].append(row[key])

In [ ]:
df_entity = pd.DataFrame(df_entity)

In [ ]:
df_entity.isna().sum()

In [ ]:
df_entity.dropna(inplace=True)

In [ ]:
df_entity.size

In [ ]:
df_entity.isna().sum()

In [ ]:
df_entity.index.name = "index"

In [ ]:
df_entity.to_csv("./data_flattened.csv")

In [ ]:
df_entity["aspect"] = df_entity["aspect"].apply(lambda row: row.lower().strip())

In [ ]:
df_entity["aspect"] = df_entity["aspect"].apply(lambda row: "speaker" if row == "speakers" else row)

In [ ]:
df_entity["aspect"] = df_entity["aspect"].apply(lambda row: "battery" if row == "battery life" else row)

In [ ]:
df_entity.aspect.unique()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_entity_train, df_entity_val = train_test_split(df_entity, test_size=0.25)

In [ ]:
df_entity_train.size, df_entity_val.size

In [ ]:
df_entity_train.to_csv("./data_train.csv")
df_entity_val.to_csv("./data_val.csv")

In [ ]:
from keyframes import GenerateKeyframes

In [ ]:
gfk = GenerateKeyframes()

In [ ]:
def get_time_range_in_sec(time):
  # get rid of unintentional spaces
  time = time.replace(" ", "")

  # get starting and ending times
  start, end = time.split("-")

  # convert to seconds
  start = int(start.split(":")[0])*60 + int(start.split(":")[1])
  end = int(end.split(":")[0])*60 + int(end.split(":")[1])

  return [start, end]

In [ ]:
def get_timestamps(data):
  # dictionary of video index as key and array of timestamps (in seconds) as value
  timestamps = {}

  for key, value in data.items():
    if "time stamp" in key.lower():
      for idx, tsp in value.items():
        if isinstance(tsp, str):
          tsp = get_time_range_in_sec(tsp)
          if idx not in timestamps:
            timestamps[idx] = [tsp]
          else:
            timestamps[idx].append(tsp)

  return timestamps

In [ ]:
timestamps = get_timestamps(df)

In [ ]:
def progress_function(self, stream, chunk, bytes_remaining):
    current = ((self.filesize - bytes_remaining)/self.filesize)
    percent = ('{0:.1f}').format(current*100)
    progress = int(50*current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar=status, percent=percent))
    sys.stdout.flush()

In [ ]:
from pytube import YouTube
from pytube.cli import on_progress 
import os

In [ ]:
for index, row in df.iterrows():
    link = row["video link"]
    file_path = f"downloads/tmp_{index}.mp4"
    yt = YouTube(link, on_progress_callback=on_progress)
    
    if not os.path.exists(f"downloads/tmp_{index}.mp4"):
        try:
            video = yt.streams.filter(res="360p")[0]
            path = video.download(filename=f"downloads/tmp_{index}.mp4")
        except:
            print(f"Error occurred while downloading video {index}. Please try for this video again later. Skipping...")
            continue
    else:
        print(f"Video {idx} exists. Skipping download...")
        path = f"downloads/tmp_{index}.mp4"
    
    # for aspect_id in range(1, 7):
    #     if pd.isna(row["aspect {}".format(aspect_id)]):
    #         continue
        # print(aspect_id)
    tsmp = timestamps[index]
        # print(tsmp)
    gfk.generateKeyframes(file_path, tsmp, index)  
    break

In [1]:
import pandas as pd
df = pd.read_csv("./data_flattened.csv", index_col="index")
df.head()

,video index,product,product name,video title,video link,time stamp,aspect,complaint label,transcript
index,,,,,,,,,
0,98,Phone,Samsung Galaxy S23 Ultra,The Samsung S23 Ultra is secretly INCREDIBLE,https://www.youtube.com/watch?v=zz70o2Ia4X0&t=1,0:28-0:38,Camera,0.0,"Plus, how have they still not managed to trim..."
1,98,Phone,Samsung Galaxy S23 Ultra,The Samsung S23 Ultra is secretly INCREDIBLE,https://www.youtube.com/watch?v=zz70o2Ia4X0&t=1,1:01-1:13,Camera,1.0,"the quality of the front camera has gone up, ..."
2,98,Phone,Samsung Galaxy S23 Ultra,The Samsung S23 Ultra is secretly INCREDIBLE,https://www.youtube.com/watch?v=zz70o2Ia4X0&t=1,1:46-1:54,OS,1.0,this thing can read and write files almost tw...
3,98,Phone,Samsung Galaxy S23 Ultra,The Samsung S23 Ultra is secretly INCREDIBLE,https://www.youtube.com/watch?v=zz70o2Ia4X0&t=1,5:09-5:24,Design,1.0,They've retained some of the curve on the scr...
4,99,Phone,iQOO 11,iQOO 11 Review with Pros & Cons | Specs on Paper,https://www.youtube.com/watch?v=2hUHgf8Hoy0,0:55-1:17,Design,1.0,is not rosy so again watch the whole video I'...


In [2]:
def get_time_range_in_sec(time):
    # get rid of unintentional spaces
    time = time.replace(" ", "")

    # get starting and ending times
    start, end = time.split("-")

    # convert to seconds
    start = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end = int(end.split(":")[0])*60 + int(end.split(":")[1])

    return [start, end]

In [ ]:
import os
import subprocess

input_folder = "./downloads/"
output_folder = "audio"

for index, row in df.iterrows():

    video_index = row["video index"]
    mp4_path = os.path.join(input_folder, f"tmp_{video_index}.mp4")
    timestamp = get_time_range_in_sec(row["time stamp"])

        # Remove the extension and add ".mp3" to get the desired output filename
    audio_filename = f"audio_{index}.mp3"

    # Build the full output path
    output_path = os.path.join(output_folder, audio_filename)

    command = [
        "ffmpeg",
        "-i", mp4_path,
        "-ss", str(timestamp[0]),
        "-to", str(timestamp[1]),
        "-vn", "-acodec", "libmp3lame",
        output_path
    ]
    # Run the command using subprocess
    subprocess.run(command)
    